# TKinter项目实战-屏保
### 项目分析
- 屏保可以自己启动，也可以手动启动
- 一旦敲击键盘或者移动鼠标后，或者其他的引发时间，则停止
- 如果屏保是一幅画的话，则没有画框
- 图像的动作是随机的，具有随机性，可能包括颜色，大小，多少， 运动方向，变形等
- 整个世界的构成是：

  - ScreenSaver:
    - 需要一个canvas， 大小与屏幕一致，没有边框
  - Ball
    - 颜色，大小，多少， 运动方向，变形等随机
    - 球能动，可以被调用

In [2]:
import random
import tkinter

In [154]:
SPEED_OF_BALL = 5

MIN_RADIUS_BALL = 20
MAX_RADIUS_BALL = 100

MIN_NUM_BALL = 5
MAX_NUM_BALL = 10


class RandomBall():
    def __init__(self, canvas, scrwidth, scrheight):
        
        # 获取主场景、和画布
        self.canvas = canvas
        
        # 定义球半径的随机大小
        self.radius = random.randint(MIN_RADIUS_BALL, MAX_RADIUS_BALL)
        
        # 定义球出现的位置
        r = self.radius
        self.x_pos = random.randint(r, int(scrwidth) - r)
        self.y_pos = random.randint(r, int(scrheight) - r)
        
        # 定义球运动的速度
        self.x_velocity = random.randint(-SPEED_OF_BALL, SPEED_OF_BALL)
        self.y_velocity = random.randint(-SPEED_OF_BALL, SPEED_OF_BALL)
        
        
        # 获取屏幕大小
        self.scrwidth = scrwidth
        self.scrheight = scrheight
        

        
        # 定义球的随机颜色
        myColor = lambda : (random.randint(0,255),random.randint(0,255),random.randint(0,255))
        self.color = '#%02x%02x%02x'%myColor()
        
        # 定义球形
        self.item = None
        
        
    def createBall(self):
        # 在canvas上画一个球
        # tkinter 只有画椭圆的函数，只需要定义长方形左上和右下两个点的坐标。
        x1 = self.x_pos - self.radius
        y1 = self.y_pos - self.radius
        x2 = self.x_pos + self.radius
        y2 = self.y_pos + self.radius
        
        # 用两个点的坐标绘制球形，fill填充颜色，outline表示外边框
        self.item = self.canvas.create_oval(x1, y1, x2, y2, fill=self.color, outline=self.color)
        
    def moveBall(self):
        # 根据速度记录一次坐标
        self.x_pos += self.x_velocity
        self.y_pos += self.y_velocity
        # 让球形移动
        self.canvas.move(self.item, self.x_velocity, self.y_velocity)
        
        # 碰撞边界检测
        x1 = self.x_pos - self.radius
        y1 = self.y_pos - self.radius
        x2 = self.x_pos + self.radius
        y2 = self.y_pos + self.radius
        
        if x1<=0 or x2>=self.scrwidth:
            self.x_velocity *= -1
        if y1<=0 or y2>=self.scrheight:
            self.y_velocity *= -1
        #print(self.scrwidth,self.scrheight,self.x_pos,self.y_pos)
        
        

class ScreenSaver():
    balls = []
    def __init__(self):
        # 定义无边框窗口
        self.root = tkinter.Tk()
        self.root.overrideredirect(1)

        # 得到显示器屏幕大小
        w,h = self.root.winfo_screenwidth(),self.root.winfo_screenheight()
        
        # 根据屏幕的大小创建画布canvas
        myColor = lambda : (random.randint(0,100),random.randint(0,100),random.randint(0,100))
        self.canvas = tkinter.Canvas(self.root,bg='#%02x%02x%02x'%myColor(), width=w, height=h)
        self.canvas.pack()
        
        # 退出：如果发生任何鼠标或键盘动作
        #self.root.bind('<Motion>', self.quitScreenSaver)
        self.root.bind("<KeyPress>", self.quitScreenSaver)
        
                
        # 每次启动产生随机数量的球
        # 在画布上放置球        
        self.randNumBalls = random.randint(MIN_NUM_BALL, MAX_NUM_BALL)
        for item in range(self.randNumBalls):
            ball = RandomBall(self.canvas, scrwidth=w, scrheight=h)
            if ball.x_velocity == ball.y_velocity == 0:
                continue
            ball.createBall()
            self.balls.append(ball)
        
        # 自动启动
        self.runScreenSaver()
        self.root.mainloop()
    
     

    # 检测一个球有没有跟其他球碰撞
    # 碰撞检测，balla一个球，ball_list所有球的列表
    def collision(self, balla, ball_list):
        x1, y1, r1 = balla.x_pos, balla.y_pos, balla.radius
        
        for ballb in ball_list:
            x2,y2,r2 = ballb.x_pos, ballb.y_pos, ballb.radius
            
            if (x2 - x1)**2 + (y2 - y1)**2 <= (r2 + r1)**2:
                temp_vx = balla.x_velocity
                temp_vy = balla.y_velocity
                balla.x_velocity = ballb.x_velocity
                balla.y_velocity = ballb.y_velocity
                ballb.x_velocity = temp_vx
                ballb.y_velocity = temp_vy
                
                
                
                
    # 启动屏保，同时开启动作循环
    def runScreenSaver(self):
        for ball_a in self.balls:
            # 球移动一步
            ball_a.moveBall()
            # 碰撞检测
            self.collision(ball_a, self.balls)
     
        # after(每隔x毫秒，启动某函数)
        # 用递归方法，创建一个时间循环
        self.canvas.after(20,self.runScreenSaver)
      
    # 退出屏保
    def quitScreenSaver(self,e): 
        self.root.destroy()

if __name__ == '__main__':
    ScreenSaver()

## 碰撞检测

In [105]:
from tkinter import *
import tkinter.messagebox
def callback(event):
    x, y = event.x, event.y
    if canvas.find_overlapping(x, y, x, y):
        msg = '(%d, %d) is in my circle!' % (x, y)
        tkinter.messagebox.askokcancel('提示',msg)

         
 
root = Tk()
canvas = Canvas(root, width=200, height=200, bg='white')
canvas.pack()
canvas.create_oval(50, 50, 150, 150, width=10)
canvas.bind('<Button-1>', callback)
root.mainloop()